# GridSearchCV Optimization for Model Variants with gen_balls
## Tối ưu hóa thông số cho các biến thể mô hình với gen_balls

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder, Binarizer
from imblearn.over_sampling import SMOTENC, SMOTEN, SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from gen_ball_mix import gen_balls
from classGBTSVM import OvO_GBTSVM, MultiLabelGBTSVM, GBTSVM, OVR_GBTSVM
from classLGBTSVM import OvO_LGBTSVM, OvR_LGBTSVM, LGBTSVM
from ucimlrepo import fetch_ucirepo
from scipy.stats import zscore
from sklearn.utils.multiclass import unique_labels
import warnings
warnings.filterwarnings('ignore')

## Định nghĩa các hàm đánh giá

In [2]:
def evaluate_base_model(model_class, train_data, test_data, *params):
    model = model_class(*params)
    model.fit(train_data)
    y_pred = model.predict(test_data)
    accuracy = model.score(test_data)
    return accuracy

def evaluate_model(model_class, X_train, y_train, X_test, y_test, *params):
    model = model_class(*params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    return accuracy

## Wrapper Classes cho các biến thể mô hình

In [3]:
class OvOGBTSVMWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, d1=0.1, d2=0.1, eps1=0.05, eps2=0.05):  # Sửa: eps1, eps2
        self.d1 = d1
        self.d2 = d2
        self.eps1 = eps1  # Đúng chính tả
        self.eps2 = eps2
        
    def fit(self, X, y):
        self.classes_ = unique_labels(y)
        self.model = OvO_GBTSVM(self.d1, self.d2, self.eps1, self.eps2)  # Sửa tham số
        self.model.fit(X, y)
        return self
        
    def predict(self, X):
        return self.model.predict(X)
        
    def score(self, X, y):
        score_percentage = self.model.score(X, y)
        return score_percentage / 100.0

class OVRGBTSVMWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, d1=0.1, d2=0.1, eps1=0.05, eps2=0.05):  # Sửa: eps1, eps2
        self.d1 = d1
        self.d2 = d2
        self.eps1 = eps1  # Đặt tên đúng
        self.eps2 = eps2  # Đặt tên đúng
    
    def fit(self, X, y):
        # Đảm bảo y là 1D array
        y = np.asarray(y).ravel()
        self.classes_ = unique_labels(y)
        self.model = OVR_GBTSVM(self.d1, self.d2, self.eps1, self.eps2)
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        # Lấy binary predictions từ OVR_GBTSVM
        binary_predictions = self.model.predict(X, threshold=0)
        
        # Chuyển đổi binary matrix thành class labels
        if binary_predictions.ndim == 2:
            # Chọn class có confidence score cao nhất
            confidence_scores = self._get_confidence_scores(X)
            predicted_indices = np.argmax(confidence_scores, axis=1)
            return self.classes_[predicted_indices]
        else:
            # Nếu đã là 1D array
            return binary_predictions
    
    def _get_confidence_scores(self, X):
        """Tính confidence scores trực tiếp từ các binary classifiers"""
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        confidence_scores = np.zeros((n_samples, n_classes))
        
        for i, cls in enumerate(self.classes_):
            model = self.model.models[cls]
            # Tạo test data với dummy label
            dummy_y = np.ones((n_samples, 1))
            test_data = np.column_stack((X, dummy_y))
            P_1 = test_data[:, :-1]
            
            # Tính confidence từ hai hyperplanes
            y1 = np.dot(P_1, model.w1) + model.b1
            y2 = np.dot(P_1, model.w2) + model.b2
            confidence = np.abs(y2) - np.abs(y1)
            confidence_scores[:, i] = confidence.flatten()
            
        return confidence_scores
    
    def score(self, X, y):
        # Đảm bảo y là 1D array
        y = np.asarray(y).ravel()
        predictions = self.predict(X)
        
        # Tính accuracy
        return np.mean(predictions == y)

        
class MultiLabelGBTSVMWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, d1=0.1, d2=0.1, eps1=0.05, eps2=0.05):  # Sửa: eps1, eps2
        self.d1 = d1
        self.d2 = d2
        self.eps1 = eps1  # Đúng chính tả
        self.eps2 = eps2
        
    def fit(self, X, y):
        self.classes_ = unique_labels(y)
        self.model = MultiLabelGBTSVM(self.d1, self.d2, self.eps1, self.eps2)  # Sửa tham số
        self.model.fit(X, y)
        return self
        
    def predict(self, X):
        return self.model.predict(X)
        
    def score(self, X, y):
        score_percentage = self.model.score(X, y)
        return score_percentage / 100.0

class OvOLGBTSVMWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, d1=0.1, d2=0.1, d3=0.1, d4=0.1):
        self.d1 = d1
        self.d2 = d2
        self.d3 = d3
        self.d4 = d4
        
    def fit(self, X, y=None):
        if y is not None:
            self.classes_ = unique_labels(y)
            data = np.column_stack([X, y])
        else:
            data = X
            self.classes_ = unique_labels(data[:, -1])
        self.model = OvO_LGBTSVM(self.d1, self.d2, self.d3, self.d4)
        self.model.fit(data)
        return self
        
    def predict(self, X):
        return self.model.predict(X)
        
    def score(self, X, y):
        data = np.column_stack([X, y])
        return self.model.score(data)

class OvRLGBTSVMWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, d1=0.1, d2=0.1, d3=0.1, d4=0.1):
        self.d1 = d1
        self.d2 = d2
        self.d3 = d3
        self.d4 = d4
        
    def fit(self, X, y=None):
        if y is not None:
            y = np.asarray(y).ravel()
            self.classes_ = unique_labels(y)
            data = np.column_stack([X, y])
        else:
            data = X
            self.classes_ = unique_labels(data[:, -1])
        
        self.n_features_train_ = X.shape[1]
        self.model = OvR_LGBTSVM(self.d1, self.d2, self.d3, self.d4)
        self.model.fit(data)
        return self
        
    def predict(self, X):
        X = np.asarray(X)
        if X.shape[1] < self.n_features_train_:
            dummy_radius = np.zeros((X.shape[0], self.n_features_train_ - X.shape[1]))
            X_processed = np.hstack([X, dummy_radius])
        elif X.shape[1] > self.n_features_train_:
            X_processed = X[:, :self.n_features_train_]
        else:
            X_processed = X
        dummy_labels = np.zeros(X_processed.shape[0])
        X_with_labels = np.column_stack([X_processed, dummy_labels])
        
        try:
            predictions = self.model.predict(X_with_labels)
            predictions = np.asarray(predictions).ravel()
            if len(predictions) != X.shape[0]:
                predictions = predictions[:X.shape[0]]
            return predictions
            
        except Exception as e:
            print(f"OvR_LGBTSVM predict error: {e}")
            return np.full(X.shape[0], self.classes_[0])
        
    def score(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).ravel()
        if X.shape[1] < self.n_features_train_:
            dummy_radius = np.zeros((X.shape[0], self.n_features_train_ - X.shape[1]))
            X_processed = np.hstack([X, dummy_radius])
        elif X.shape[1] > self.n_features_train_:
            X_processed = X[:, :self.n_features_train_]
        else:
            X_processed = X
        
        try:
            data_with_labels = np.column_stack([X_processed, y])
            score_percentage = self.model.score(data_with_labels)
            return score_percentage / 100.0
            
        except Exception as e:
            print(f"OvR_LGBTSVM score error: {e}")
            try:
                predictions = self.predict(X)
                return np.mean(predictions == y)
            except:
                return 0.0

## Định nghĩa Parameter Grids

In [4]:
# Parameter grids cho từng mô hình
gbtsvm_params = {
    'd1': [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5],
    'd2': [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5],
    'eps1': [0.01, 0.025, 0.05, 0.075, 0.1],
    'eps2': [0.01, 0.025, 0.05, 0.075, 0.1]
}

lgbtsvm_params = {
    'd1': [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5],
    'd2': [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5],
    'd3': [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5],
    'd4': [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5]
}

# Parameters cho gen_balls
gen_balls_params = {
    'pur': [1, 2, 5, 7, 10],
    'num': [1, 2, 3, 4, 5]
}


## Hàm tối ưu hóa cho các biến thể với gen_balls

In [5]:
def optimize_variants_with_gen_balls_final(variant_models, original_data, test_ratio=0.2, cv=3):
    all_results = {}
    
    # Tách dữ liệu gốc
    m = original_data.shape[0]
    train_size = int(m * (1 - test_ratio))
    
    # Lưu dữ liệu test gốc
    original_test_data = original_data[train_size:]
    X_test_original = original_test_data[:, :-1]
    y_test_original = original_test_data[:, -1].ravel()
    
    for pur in gen_balls_params['pur']:
        for num in gen_balls_params['num']:
            print(f"\n{'='*60}")
            print(f"Testing gen_balls: pur={1 - (0.015 * pur)}, num={num}")
            print(f"{'='*60}")
            
            try:
                # Áp dụng gen_balls
                original_train_data = original_data[:train_size]
                train_data_processed = gen_balls(original_train_data, pur=1 - (0.015 * pur), delbals=num)
                
                if len(train_data_processed) == 0:
                    print("Gen_balls returned empty result, skipping...")
                    continue
                
                # Xử lý dữ liệu gen_balls output
                Radius = [i[1] for i in train_data_processed]
                Center = [i[0] for i in train_data_processed]
                Label = [i[-1] for i in train_data_processed]
                    
                Radius = np.array(Radius)
                Center = np.array(Center)
                Label = np.array(Label)
                
                Z_train = np.hstack((Center, Radius.reshape(Radius.shape[0], 1)))
                Lab = Label.reshape(Label.shape[0], 1)
                train_data_final = np.hstack((Z_train, Lab))
                
                # Chuẩn bị dữ liệu
                X_train_processed = train_data_final[:, :-1]
                y_train_processed = train_data_final[:, -1].ravel()
                X_test = X_test_original
                y_test = y_test_original
                
                print(f"X_train_processed shape: {X_train_processed.shape}")
                print(f"y_train_processed shape: {y_train_processed.shape}")
                print(f"X_test shape: {X_test.shape}")
                print(f"y_test shape: {y_test.shape}")
                
                # Kiểm tra dữ liệu hợp lệ
                if X_train_processed.shape[0] < 10:
                    print("Too few training samples, skipping...")
                    continue
                
                # Tối ưu hóa từng biến thể
                for variant_name, (model_class, param_grid) in variant_models.items():
                    print(f"\nOptimizing {variant_name}...")
                    
                    try:
                        grid_search = GridSearchCV(
                            estimator=model_class(),
                            param_grid=param_grid,
                            cv=min(cv, 3),  # Giảm CV folds nếu cần
                            scoring='accuracy',
                            n_jobs=1,
                            verbose=0
                        )
                        
                        grid_search.fit(X_train_processed, y_train_processed)
                        test_score = grid_search.score(X_test, y_test)
                        
                        # Lưu kết quả
                        result_key = f"{variant_name}_pur{pur}_num{num}"
                        all_results[result_key] = {
                            'model_name': variant_name,
                            'gen_balls_pur': pur,
                            'gen_balls_num': num,
                            'best_model_params': grid_search.best_params_,
                            'cv_score': grid_search.best_score_,
                            'test_score': test_score,
                            'best_estimator': grid_search.best_estimator_
                        }
                        
                        print(f"Best params: {grid_search.best_params_}")
                        print(f"CV Score: {grid_search.best_score_:.4f}")
                        print(f"Test Score: {test_score:.4f}")
                        
                    except Exception as model_error:
                        print(f"Error optimizing {variant_name}: {str(model_error)}")
                        continue
                    
            except Exception as e:
                print(f"Error with pur={pur}, num={num}: {str(e)}")
                continue
    
    return all_results


## Hàm tạo bảng so sánh kết quả

In [6]:
def create_results_summary(all_results):
    """
    Tạo bảng tổng hợp kết quả
    """
    summary_data = []
    
    for result_key, results in all_results.items():
        summary_data.append({
            'Model': results['model_name'],
            'Gen_Balls_Pur': results['gen_balls_pur'],
            'Gen_Balls_Num': results['gen_balls_num'],
            'CV_Score': results['cv_score'],
            'Test_Score': results['test_score'],
            'Best_Params': str(results['best_model_params'])
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('Test_Score', ascending=False)
    
    return summary_df

def find_best_for_each_model(all_results):
    """
    Tìm kết quả tốt nhất cho từng mô hình
    """
    model_best = {}
    
    for result_key, results in all_results.items():
        model_name = results['model_name']
        test_score = results['test_score']
        
        if model_name not in model_best or test_score > model_best[model_name]['test_score']:
            model_best[model_name] = results
    
    return model_best

## Tải và chuẩn bị dữ liệu

In [24]:
# Tải dữ liệu
id_dataset = 39
dermatology = fetch_ucirepo(id=33) 
X = dermatology.data.features 
y = dermatology.data.targets.squeeze()
X_df = pd.DataFrame(X)
X_clean = X_df.dropna(subset=['age'])
y_clean = y[X_clean.index]

# Q1 = np.percentile(X, 25, axis=0)
# Q3 = np.percentile(X, 75, axis=0)
# IQR = Q3 - Q1
# mask_outlier = np.all((X >= Q1 - 1.5*IQR) & (X <= Q3 + 1.5*IQR), axis=1)

# X = X[mask_outlier]
# y = y[mask_outlier]

# # Loại bỏ NaN trong y
# mask_nan = ~np.isnan(y) if y.dtype.kind in ['f', 'i'] else (y != '') & (y != None)
# X_clean = X[mask_nan]
# y_clean = y[mask_nan]

# Áp dụng SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_clean, y_clean)

# Tạo DataFrame và encode target
df = pd.concat([X_res, y_res], axis=1)
target_col = y.name
enc_target = LabelEncoder()
df[target_col] = enc_target.fit_transform(df[target_col]).astype(np.float64)

# Chuyển thành numpy array
data = df.to_numpy(dtype=np.float64)
m, n = data.shape
#for Global Cancer
# data = data[0:int(m*(1-0.98))]
# m,n = data.shape

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(-1, 1))
data[:, :-1] = scaler.fit_transform(data[:, :-1])

# Shuffle dữ liệu
np.random.seed(0)
indices = np.random.permutation(m)
data = data[indices]

print(f"Data shape: {data.shape}")
print(f"Number of classes: {len(np.unique(data[:, -1]))}")

Data shape: (666, 35)
Number of classes: 6


## Chạy baseline với thông số mặc định

In [25]:
# Chia train/test
A_train = data[0:int(m*(1-0.20))]
A_test = data[int(m * (1-0.20)):]

# Áp dụng gen_balls với thông số mặc định
pur = 1 - (0.015 * 5) 
num = 2
A_train_gen = gen_balls(A_train, pur=pur, delbals=num)

# Xử lý output của gen_balls
Radius = []
for i in A_train_gen:
    Radius.append(i[1])
Center = []
for ii in A_train_gen:
    Center.append(ii[0])
Label = []
for iii in A_train_gen:
    Label.append(iii[-1])
    
Radius = np.array(Radius)
Center = np.array(Center)
Label = np.array(Label)
Z_train = np.hstack((Center, Radius.reshape(Radius.shape[0], 1)))
Lab = Label.reshape(Label.shape[0], 1)
A_train_processed = np.hstack((Z_train, Lab))

# Tách features và labels
X_train = A_train_processed[:,:-1]
X_test = A_test[:,:-1]
y_train = A_train_processed[:,-1]
y_test = A_test[:,-1]

print(f"Training data shape: {A_train.shape}")
print(f"Processed training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (532, 35)
Processed training data shape: (19, 35)
Test data shape: (134, 34)


## Đánh giá baseline models

In [26]:
import time

# Đánh giá các mô hình với thông số mặc định và tính thời gian thực hiện
print("=== BASELINE RESULTS ===")

start_time = time.time()
accuracy_ovo_gbtsvm = evaluate_model(OvO_GBTSVM, X_train, y_train, X_test, y_test, 0.1, 0.1, 0.05, 0.05)
end_time = time.time()
print(f"Accuracy of OvO_GBTSVM: {accuracy_ovo_gbtsvm:.4f}")
print(f"Time taken for OvO_GBTSVM: {end_time - start_time:.4f} seconds")

start_time = time.time()
accuracy_ovr_gbtsvm = evaluate_model(OVR_GBTSVM, X_train, y_train, X_test, y_test, 0.1, 0.1, 0.05, 0.05)
end_time = time.time()
print(f"Accuracy of OVR_GBTSVM: {accuracy_ovr_gbtsvm:.4f}")
print(f"Time taken for OVR_GBTSVM: {end_time - start_time:.4f} seconds")

start_time = time.time()
accuracy_multi_gbtsvm = evaluate_model(MultiLabelGBTSVM, X_train, y_train, X_test, y_test, 0.1, 0.1, 0.05, 0.05)
end_time = time.time()
print(f"Accuracy of MultiLabelGBTSVM: {accuracy_multi_gbtsvm:.4f}")
print(f"Time taken for MultiLabelGBTSVM: {end_time - start_time:.4f} seconds")

start_time = time.time()
accuracy_ovo_lgbtsvm = evaluate_base_model(OvO_LGBTSVM, A_train_processed, A_test, 0.1, 0.1, 0.1, 0.1)
end_time = time.time()
print(f"Accuracy of OvO_LGBTSVM: {accuracy_ovo_lgbtsvm:.4f}")
print(f"Time taken for OvO_LGBTSVM: {end_time - start_time:.4f} seconds")

start_time = time.time()
accuracy_ovr_lgbtsvm = evaluate_base_model(OvR_LGBTSVM, A_train_processed, A_test, 0.1, 0.1, 0.1, 0.1)
end_time = time.time()
print(f"Accuracy of OvR_LGBTSVM: {accuracy_ovr_lgbtsvm:.4f}")
print(f"Time taken for OvR_LGBTSVM: {end_time - start_time:.4f} seconds")


=== BASELINE RESULTS ===
Accuracy of OvO_GBTSVM: 94.0299
Time taken for OvO_GBTSVM: 0.1075 seconds
Accuracy of OVR_GBTSVM: 70.8955
Time taken for OVR_GBTSVM: 0.0708 seconds
Accuracy of MultiLabelGBTSVM: 84.3284
Time taken for MultiLabelGBTSVM: 0.4103 seconds
Accuracy of OvO_LGBTSVM: 70.1493
Time taken for OvO_LGBTSVM: 0.1018 seconds
Accuracy of OvR_LGBTSVM: 39.5522
Time taken for OvR_LGBTSVM: 0.0963 seconds


## Tối ưu hóa các biến thể mô hình với gen_balls

In [ ]:
# Định nghĩa các biến thể cần tối ưu hóa
variant_models = {
    'OvO_GBTSVM': (OvOGBTSVMWrapper, gbtsvm_params),
    'OVR_GBTSVM': (OVRGBTSVMWrapper, gbtsvm_params)
    # 'MultiLabel_GBTSVM': (MultiLabelGBTSVMWrapper, gbtsvm_params),
    # 'OvO_LGBTSVM': (OvOLGBTSVMWrapper, lgbtsvm_params),
    # 'OvR_LGBTSVM': (OvRLGBTSVMWrapper, lgbtsvm_params)
}

print("\n=== OPTIMIZING VARIANT MODELS WITH GEN_BALLS ===")
optimization_results = optimize_variants_with_gen_balls_final(
    variant_models=variant_models,
    original_data=data,
    test_ratio=0.2,
    cv=3
)

## Tạo bảng tổng hợp kết quả

In [ ]:
# Tạo bảng tổng hợp
print("\n=== SUMMARY OF ALL RESULTS ===")
summary_table = create_results_summary(optimization_results)
print(summary_table.to_string(index=False))

# Tìm kết quả tốt nhất cho từng mô hình
print("\n=== BEST RESULTS FOR EACH MODEL ===")
best_results = find_best_for_each_model(optimization_results)

for model_name, results in best_results.items():
    print(f"\n{model_name}:")
    print(f"  Best Test Score: {results['test_score']:.4f}")
    print(f"  Gen_balls params: pur={results['gen_balls_pur']}, num={results['gen_balls_num']}")
    print(f"  Model params: {results['best_model_params']}")

## Tìm mô hình tổng thể tốt nhất

In [ ]:
# Tìm mô hình tốt nhất tổng thể
best_overall = max(optimization_results.items(), key=lambda x: x[1]['test_score'])
best_key, best_result = best_overall

print(f"\n=== BEST OVERALL MODEL ===")
print(f"Model: {best_result['model_name']}")
print(f"Test Score: {best_result['test_score']:.4f}")
print(f"CV Score: {best_result['cv_score']:.4f}")
print(f"Gen_balls parameters: pur={best_result['gen_balls_pur']}, num={best_result['gen_balls_num']}")
print(f"Model parameters: {best_result['best_model_params']}")

## Lưu kết quả

In [ ]:
# Lưu bảng tổng hợp
summary_table.to_csv(f'{id_dataset}_GTBSVM_variant_models_optimization_results.csv', index=False)
print("\nKết quả tổng hợp đã được lưu vào 'variant_models_optimization_results.csv'")

# Tạo bảng kết quả tốt nhất cho từng mô hình
best_results_data = []
for model_name, results in best_results.items():
    best_results_data.append({
        'Model': model_name,
        'Best_Test_Score': results['test_score'],
        'Best_CV_Score': results['cv_score'],
        'Best_Gen_Balls_Pur': results['gen_balls_pur'],
        'Best_Gen_Balls_Num': results['gen_balls_num'],
        'Best_Model_Params': str(results['best_model_params'])
    })

best_results_df = pd.DataFrame(best_results_data)
best_results_df = best_results_df.sort_values('Best_Test_Score', ascending=False)
best_results_df.to_csv(f'{id_dataset}_GTBSVM_best_results_per_model.csv', index=False)
print("Kết quả tốt nhất cho từng mô hình đã được lưu vào 'best_results_per_model.csv'")

# Lưu kết quả chi tiết
import pickle
with open(f'{id_dataset}_GTBSVM_detailed_optimization_results.pkl', 'wb') as f:
    pickle.dump({
        'all_results': optimization_results,
        'best_results': best_results,
        'best_overall': best_result,
        'summary_table': summary_table
    }, f)
print(f"Kết quả chi tiết đã được lưu vào '{id_dataset}_GTBSVM_optimization_results.pkl'")